In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("GSAF5.csv", encoding='latin1')


# Check data shape, head and columns
#df.shape
#df.head()
#df.columns



In [4]:
# 1.Renommer les colonnes en ajoutant les '_'
df_rename_columns = df.rename(columns = {'Case Number':'Case_number','Investigator or Source':'Investigator_or_Source','href formula':'href_formula','Case Number.1':'Case_number1','Case Number.2':'Case_number2','original order':'original_order'})
#df_rename_columns.columns

In [5]:
# 2. Uniformiser les noms de colonnes : première lettre en Maj
df_rename_columns.columns = df_rename_columns.columns.str.capitalize()
#df_rename_columns


In [6]:
# 3. Changer l'ordre des colonnes
column_order = ['Case_number','Case_number1', 'Case_number2', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (y/n)', 'Time',
       'Species ', 'Investigator_or_source', 'Pdf', 'Href_formula', 'Href',
        'Original_order', 'Unnamed: 22',
       'Unnamed: 23']
df_rename_columns = df_rename_columns[column_order]
#df_rename_columns.head()

In [7]:
# 4. Check colonne Name : dès les première lignes on lit 'male' et 'female'.
# correction de male et female = none

df_rename_columns['Name']
df_rename_columns.loc[(df_rename_columns['Name'])=='male', 'Name']='None'
df_rename_columns.loc[(df_rename_columns['Name'])=='female', 'Name']='None'
#df_rename_columns['Name']

In [8]:
# 5. Supprimer les colonnes vides: colonnes Unnamed 22 et 23
df_rename_columns.isnull().sum()
df_drop_columns = df_rename_columns.drop(df_rename_columns.columns[[22,23]], axis='columns')
#df_drop_columns

In [9]:
# 6.Missind value
df_drop_columns.isnull().sum()

Case_number                  0
Case_number1                 0
Case_number2                 0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (y/n)                 19
Time                      3213
Species                   2934
Investigator_or_source      15
Pdf                          0
Href_formula                 1
Href                         3
Original_order               0
dtype: int64

In [10]:
# Remplacer les valeurs nulles par 0
df_drop_columns[['Country','Area','Location','Activity','Name','Sex ','Age','Injury','Fatal (y/n)','Time','Species','Investigator_or_source']] = df_drop_columns[['Country','Area','Location','Activity','Name','Sex ','Age','Injury','Fatal (y/n)','Time','Species ','Investigator_or_source']].fillna('0')
df_drop_columns.isnull().sum()
#df_drop_columns

Case_number                  0
Case_number1                 0
Case_number2                 0
Date                         0
Year                         0
Type                         0
Country                      0
Area                         0
Location                     0
Activity                     0
Name                         0
Sex                          0
Age                          0
Injury                       0
Fatal (y/n)                  0
Time                         0
Species                   2934
Investigator_or_source       0
Pdf                          0
Href_formula                 1
Href                         3
Original_order               0
Species                      0
dtype: int64

In [12]:
# 7. Retrouver et supprimer les doublons

# sur la table entière
before = len(df_drop_columns)
df_drop_columns = df_drop_columns.drop_duplicates()
after = len(df_drop_columns)
print('Number of duplicate records dropped: ', str(before - after))

# sur les colonnes case number, case number 1 et 2
select_columns = ['Case_number','Case_number1', 'Case_number2']

df_drop_columns = df_drop_columns.drop_duplicates(subset=select_columns)
after = len(df_drop_columns)
print('Number of duplicate records dropped: ', str(before - after))

#df_drop_columns

Number of duplicate records dropped:  0
Number of duplicate records dropped:  0


In [13]:
# 8. Vérifier si il y a des doublons ou erreurs (infos: nous avons 5992 entrées et 22 colonnes)

"""Comparaison des colonnes:
df_drop_columns[df_drop_columns["Case_number"]!=df_drop_columns["Case_number1"]]
on detecte 13 erreurs sur 5992 entrées : erreur de frappe

df_drop_columns[df_drop_columns["Case_number1"]!=df_drop_columns["Case_number2"]]
on detecte 11 erreurs sur 5992 entrées : erreur de frappe

df_drop_columns[df_drop_columns["Case_number"]!=df_drop_columns["Case_number2"]]
on detecte 2 erreurs sur 5992 entrées : erreur de frappe
Je choisis de corriger les erreurs en remplaçant les valeurs incorrects"""



'Comparaison des colonnes:\ndf_drop_columns[df_drop_columns["Case_number"]!=df_drop_columns["Case_number1"]]\non detecte 13 erreurs sur 5992 entrées : erreur de frappe\n\ndf_drop_columns[df_drop_columns["Case_number1"]!=df_drop_columns["Case_number2"]]\non detecte 11 erreurs sur 5992 entrées : erreur de frappe\n\ndf_drop_columns[df_drop_columns["Case_number"]!=df_drop_columns["Case_number2"]]\non detecte 2 erreurs sur 5992 entrées : erreur de frappe\nJe choisis de corriger les erreurs en remplaçant les valeurs incorrects'

In [14]:
# correction des erreurs

corr = df_drop_columns

# filtre contenant les erreurs uniquement

filt = corr['Case_number'] != corr['Case_number1']
filt = filt | (corr['Case_number2'] != corr['Case_number1'])
filt = filt| (corr['Case_number2'] != corr['Case_number'])
differences = corr[filt]

t1=pd.get_dummies(differences['Case_number'])
t2=pd.get_dummies(differences['Case_number1'])
t3=pd.get_dummies(differences['Case_number2'])

columns = list(set(list(t1.columns) + list(t2.columns) + list(t3.columns)))
#print(columns)
for c in columns:
    if c not in t1.columns:
        t1[c] = 0
    if c not in t2.columns:
        t2[c] = 0
    if c not in t3.columns:
        t3[c] = 0

t1 = t1[columns]
t2 = t2[columns]
t3 = t3[columns]

sum_t = t1+t2+t3

sum_t.idxmax(axis=1)

# nommer la colonne d'erreurs

modif = sum_t.idxmax(axis=1)

In [15]:
# intégrer les données corrigées au tableau 

modif = sum_t.idxmax(axis=1)

df_drop_columns.loc[modif.index, 'Case_number'] = modif
df_drop_columns.loc[modif.index, 'Case_number1'] = modif
df_drop_columns.loc[modif.index, 'Case_number2'] = modif
#df_drop_columns

In [14]:
# Revoir la colonne Date
# split en 4 colonnes : JJ/MM/AA/Commentaires

In [22]:
#df_drop_columns['Date'].value_counts(dropna=False)


In [22]:
import re

column = df_drop_columns['Date']
colonne_commentaire = []
colonne_jour = []
colonne_mois = []
colonne_annee = []

for x in column:
    if ma

In [21]:
column = df_drop_columns['Date']
colonne_commentaire = []
colonne_jour = []
colonne_mois = []
colonne_annee = []

for x in column:
    # test si cest une date Normale (2chiffres-3lettres-2chiffres)
        # met les valeurs de jour mois annee au bon endroit
        # et met None dans commentaire
        
    # sinon si cest pas une date Normale
        # test si cest Reported + une date
        # si oui : tu mets reported dans commentaire et la date au bon endroit
    
    # plsuierus sinon qui traitent tous les cas
    
    # else a la fin : si on a rien trouve
        #on met None partout
    
    if x.startswith("Reported"):
        colonne_commentaire.append('Reported')
        if x.("{:\d{4}\.\d{2}\.\d{2}\}")
            colonne_jour.append("{:\d2}")
            
    else:
        colonne_commentaire.append(None)

colonne_commentaire
        

SyntaxError: invalid syntax (<ipython-input-21-ccb74a620c7b>, line 23)

In [1]:
#check type de Data
#df_drop_columns['Date'] = df_drop_columns['Date'].astype('date')
#df_drop_columns['Date'].dtype

# styler
#df_drop_columns['Date']
#df_drop_columns.style.format("{:\d{4}\.\d{2}\.\d{2}\.?[a-zA-Z]?'}", subset='Date')



column = df_drop_columns['Date']
colonne_commentaire = []
colonne_jour = []
colonne_mois = []
colonne_annee = []

for x in column:
    # test si cest une date Normale (2chiffres-3lettres-2chiffres)
        # met les valeurs de jour mois annee au bon endroit
        # et met None dans commentaire
        
    # sinon si cest pas une date Normale
        # test si cest Reported + une date
        # si oui : tu mets reported dans commentaire et la date au bon endroit
    
    # plsuierus sinon qui traitent tous les cas
    
    # else a la fin : si on a rien trouve
        #on met None partout
    
    if x.startswith("Reported"):
        val_comm.append('Reported')
    else:
        val_comm.append(None)
val_comm
        
        


NameError: name 'df_drop_columns' is not defined

In [17]:
#df_drop_columns.Date=('\d{4}\.\d{2}\.\d{2}\.?[a-zA-Z]?')
#df_drop_columns['Date']

In [18]:
data = df_drop_columns.Date.str.replace(' ','-')
data = data.str.replace('Jan','01')
data = data.str.replace('Feb','02')
data = data.str.replace('Mar','03')
data = data.str.replace('Apr','04')
data = data.str.replace('May','05')
data = data.str.replace('Jun','06')
data = data.str.replace('Jul','07')
data = data.str.replace('Aug','08')
data = data.str.replace('Sep', '09')
data = data.str.replace('Oct', '10')
data = data.str.replace('Nov','11')
data = data.str.replace('Dec','12')

data = data.str.split("\W",n=3, expand=True)
data



,0,1,2,3
0,18,09,16,None
1,18,09,16,None
2,18,09,16,None
3,17,09,16,None
4,16,09,16,None
5,15,09,16,None
6,11,09,16,None
7,07,09,16,None
8,06,09,16,None
9,05,09,16,None


In [19]:
# Dans la colonne Date : supprimer le mot before pour ne conserver que les années

data = df_drop_columns.Date.str.replace('Sep','09')
data = data.str.replace('Aug', '08')
data = data.str.replace('Jul', '07')
#data = data.str.replace('[Aa-zZ]','')
#data = data.str.replace('\D','')
#data = data.str.replace('\d+-\d+-\d+','' )
#data = data.str.replace('\d+-\d+','' )
#data = data.str.replace('\w','' )
data = data.str.replace('[Aa-zZ]','' )
data 

0                        18-09-16
1                        18-09-16
2                        18-09-16
3                        17-09-16
4                        16-09-16
5                        15-09-16
6                        11-09-16
7                        07-09-16
8                        06-09-16
9                        05-09-16
10                       05-09-16
11                       04-09-16
12                       01-09-16
13                       29-08-16
14                       29-08-16
15                       27-08-16
16                       25-08-16
17                       07-08-16
18                       06-08-16
19                       04-08-16
20                       29-07-16
21                       28-07-16
22                       28-07-16
23                       27-07-16
24                       26-07-16
25                       24-07-16
26                       23-07-16
27                       23-07-16
28                       20-07-16
29            